# TensorBoard

After Training the model, run 

    tensorboard --logdir=path/to/log-directory
    
Tensorboard provides a good visualization tool for all the variables you like and works on a browser.

In [2]:
import tensorflow as tf
import input_data

In [3]:
def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name=name)

# This network is the same as the previous one except with an extra hidden layer + dropout
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    # Add layer name scopes for better graph visualization
    with tf.name_scope("layer1"):
        X = tf.nn.dropout(X, p_keep_input)
        h = tf.nn.relu(tf.matmul(X, w_h))
    with tf.name_scope("layer2"):
        h = tf.nn.dropout(h, p_keep_hidden)
        h2 = tf.nn.relu(tf.matmul(h, w_h2))
    with tf.name_scope("layer3"):
        h2 = tf.nn.dropout(h2, p_keep_hidden)
        return tf.matmul(h2, w_o)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Succesfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


/usr/lib/python3.5/gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/home/lifefeel/TensorFlowTutorials/input_data.py:42: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Succesfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Succesfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Succesfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
X = tf.placeholder("float", [None, 784], name="X")
Y = tf.placeholder("float", [None, 10], name="Y")

w_h = init_weights([784, 625], "w_h")
w_h2 = init_weights([625, 625], "w_h2")
w_o = init_weights([625, 10], "w_o")

# Add histogram summaries for weights
tf.histogram_summary("w_h_summ", w_h)
tf.histogram_summary("w_h2_summ", w_h2)
tf.histogram_summary("w_o_summ", w_o)

p_keep_input = tf.placeholder("float", name="p_keep_input")
p_keep_hidden = tf.placeholder("float", name="p_keep_hidden")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
    train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    # Add scalar summary for cost
    tf.scalar_summary("cost", cost)

with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(py_x, 1)) # Count correct predictions
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float")) # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.scalar_summary("accuracy", acc_op)

In [5]:
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()

    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
        summary, acc = sess.run([merged, acc_op], feed_dict={X: teX, Y: teY,
                                          p_keep_input: 1.0, p_keep_hidden: 1.0})
        writer.add_summary(summary, i)  # Write summary
        print(i, acc)                   # Report the accuracy

0 0.9284
1 0.9609
2 0.9724
3 0.974
4 0.9763
5 0.977
6 0.9781
7 0.9802
8 0.9788
9 0.9783
10 0.9786
11 0.9784
12 0.9813
13 0.9807
14 0.9808
15 0.9806
16 0.9816
17 0.9814
18 0.9827
19 0.9818
20 0.9829
21 0.982
22 0.983
23 0.9821
24 0.9833
25 0.9836
26 0.9832
27 0.9831
28 0.9836
29 0.9824
30 0.9831
31 0.9833
32 0.9817
33 0.9838
34 0.9836
35 0.9839
36 0.9831
37 0.9828
38 0.9837
39 0.983
40 0.9826
41 0.9831
42 0.9837
43 0.9843
44 0.9837
45 0.984
46 0.9837
47 0.9843
48 0.9839
49 0.9846
50 0.9843
51 0.9843
52 0.9834
53 0.9855
54 0.9839
55 0.9848
56 0.9838
57 0.9844
58 0.9828
59 0.9838
60 0.9844
61 0.9828
62 0.9839
63 0.9842
64 0.9841
65 0.9847
66 0.984
67 0.9847
68 0.9833
69 0.9841
70 0.9846
71 0.9852
72 0.9845
73 0.9852
74 0.985
75 0.9848
76 0.9845
77 0.9853
78 0.9837
79 0.984
80 0.985
81 0.9845
82 0.9842
83 0.9846
84 0.9845
85 0.9846
86 0.9853
87 0.985
88 0.9851
89 0.9847
90 0.9841
91 0.9844
92 0.9844
93 0.9861
94 0.984
95 0.9842
96 0.986
97 0.9854
98 0.9851
99 0.9861
